### Przygotowanie danych

In [2]:
# pip install requirements.txt

In [121]:
import pandas as pd #tabele
import numpy as np #np. wektory, listy
import matplotlib.pyplot as plt #wykresy
import seaborn as sns #wykresy korelacji
from scipy import stats #kruskal-wallis, mann-whitney
from sklearn.impute import KNNImputer
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyClassifier
#from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
import missingno as msno
from sktime.transformations.series.impute import Imputer
import plotly.express as px
import warnings

In [250]:
df0 = pd.read_excel("..\\gasometry_data\\dane_surowe_gaz.xlsx", sheet_name="Arkusz1")

In [5]:
df0=df0.pivot_table(index=("PACJENT_NR", "BADANIE_NR", "ZGON"), columns=["KOD_BADANIA"], values=["WYNIK"]) #ustawienie wyników w kolumnach
df0.reset_index(inplace=True) #rozpakowanie indeksów
df0.columns.name="Lp"
df0.columns=['PACJENT_NR','BADANIE_NR', 'ZGON', 'BETET', 'CO2TET', 'HCO3ACTE', 'HCO3STTE', 'O2SATTET', 'O2TET', 'PHTET']
#zamiana PH na liczbę jonów wodoru, min:35 , max: 45 (im więcej, tym niższe PH)
df0["IONH"]=10**(9-df0["PHTET"])

In [6]:
df0.head()

,PACJENT_NR,BADANIE_NR,ZGON,BETET,CO2TET,HCO3ACTE,HCO3STTE,O2SATTET,O2TET,PHTET,IONH
0,1,1,NIE,-0.8,57.1,26.1,21.9,82.6,57.0,7.269,53.826978
1,1,2,NIE,-1.9,87.2,26.2,21.8,99.3,197.0,NaN,NaN
2,1,3,NIE,1.2,66.0,28.5,24.2,99.6,225.0,7.244,57.016427
3,1,4,NIE,1.7,62.5,28.6,23.7,98.1,111.0,7.269,53.826978
4,1,5,NIE,0.5,61.1,27.5,23.9,99.1,145.0,7.262,54.701596


### Filtrowanie pacjentów >6 pomiarów

In [7]:
id_df=df0[["PACJENT_NR","BADANIE_NR"]].groupby('PACJENT_NR').max().add_suffix('_liczba')>6
id6=id_df.index[id_df["BADANIE_NR_liczba"]]
df_filtr=df0[df0['PACJENT_NR'].isin(id6)]
df_filtr

,PACJENT_NR,BADANIE_NR,ZGON,BETET,CO2TET,HCO3ACTE,HCO3STTE,O2SATTET,O2TET,PHTET,IONH
0,1,1,NIE,-0.8,57.1,26.1,21.9,82.6,57.0,7.269,53.826978
1,1,2,NIE,-1.9,87.2,26.2,21.8,99.3,197.0,NaN,NaN
2,1,3,NIE,1.2,66.0,28.5,24.2,99.6,225.0,7.244,57.016427
3,1,4,NIE,1.7,62.5,28.6,23.7,98.1,111.0,7.269,53.826978
4,1,5,NIE,0.5,61.1,27.5,23.9,99.1,145.0,7.262,54.701596
...,...,...,...,...,...,...,...,...,...,...,...
3040,145,5,TAK,-5.6,51.3,21.8,19.6,95.9,85.5,7.237,57.942870
3041,145,6,TAK,-6.4,48.9,20.9,19.2,98.8,116.0,7.240,57.543994
3042,145,7,TAK,-7.2,52.1,20.7,18.4,94.7,82.9,7.207,62.086903
3043,145,8,TAK,-5.5,46.5,21.4,19.9,98.1,115.0,7.271,53.579666


In [253]:
df_pom=df_filtr.copy()
df_pom["BETET"] = (df_pom["BETET"] - (-2.3))/(2.3 - -(2.3))
df_pom["IONH"] = (df_pom["IONH"] - 35)/(45 - 35)
df_pom["O2SATTET"] = (df_pom["O2SATTET"] - 95)/(100 - 95)
df_pom["CO2TET"] = (df_pom["CO2TET"] - 35)/(45 - 35)
df_pom["O2TET"] = (df_pom["O2TET"] - 75)/(100 - 75)
df_pom["HCO3STTE"] = (df_pom["HCO3STTE"] - 21)/(25 - 21)
df_pom["HCO3ACTE"] = (df_pom["HCO3ACTE"] - 21)/(27 - 21)
df_pom["PHTET"] = (df_pom["PHTET"] - 7.35)/(7.45 - 7.35)
df_pom["ZGON"] = np.where(df_pom["ZGON"] == "NIE", 0, 1)

df_filtr_std=df_pom
df_filtr_std.to_csv("filtered_std_data_gasometry.csv")

### Poszukiwanie najlepszej metody uzupełniania braków danych

In [ ]:
#Działanie na danych przefiltrowanych, ale nieujednoliconych
# df_na=df_filtr.copy()
# cols=["BETET",	"CO2TET",	"HCO3ACTE",	"HCO3STTE",	"O2SATTET",	"O2TET", "IONH"]
# df_na["czy_na"]=df_na[cols].apply(lambda row: any(row.isna()), axis=1)
# id=df_na[df_na["czy_na"]==True]["PACJENT_NR"]
# df_bezna = df_na[~df_na['PACJENT_NR'].isin(id)]
# df_bezna = df_bezna[["PACJENT_NR", *cols]]
# df_bezna
#len(df_na[df_na["czy_na"]==True].index)
#max_values = df_na.groupby('PACJENT_NR')['BADANIE_NR'].max()
# print(max_values[id])
# print(df_na[df_na["czy_na"]==True])

In [200]:
#Działanie na przefiltrowanych i ujednolicenie zmiennych 
df_imp=df_filtr_std.copy()
cols=["BETET",	"CO2TET",	"HCO3ACTE",	"HCO3STTE",	"O2SATTET",	"O2TET", "IONH"]
df_imp["czy_na"]=df_imp[cols].apply(lambda row: any(row.isna()), axis=1)
id=df_imp[df_imp["czy_na"]==True]["PACJENT_NR"]
df_bezna = df_imp[~df_imp['PACJENT_NR'].isin(id)]
df_bezna = df_bezna[["PACJENT_NR", *cols]]
df_bezna

,PACJENT_NR,BETET,CO2TET,HCO3ACTE,HCO3STTE,O2SATTET,O2TET,IONH
50,2,-0.043478,3.22,0.983333,0.175,-1.52,-0.372,2.525596
51,2,0.739130,2.90,1.183333,0.675,0.16,0.868,1.982770
52,2,1.086957,3.73,1.516667,0.900,0.38,1.364,2.294287
53,2,0.739130,3.47,1.566667,0.875,0.02,0.572,2.033501
54,2,1.108696,2.23,1.516667,1.300,0.46,1.032,1.081419
...,...,...,...,...,...,...,...,...
3040,145,-0.717391,1.63,0.133333,-0.350,0.18,0.420,2.294287
3041,145,-0.891304,1.39,-0.016667,-0.450,0.76,1.640,2.254399
3042,145,-1.065217,1.71,-0.050000,-0.650,-0.06,0.316,2.708690
3043,145,-0.695652,1.15,0.066667,-0.275,0.62,1.600,1.857967


In [201]:
np.random.seed(13)
num_values_to_remove =int(1000)
rows_to_remove = np.random.choice(df_bezna.index, size=num_values_to_remove, replace=True)
cols_to_remove = np.random.choice(df_bezna.columns[1:10], size=num_values_to_remove, replace=True)
id=zip(rows_to_remove, cols_to_remove)
df_nowe=df_bezna.copy()

for row, col in id:
  df_nowe.loc[row, col] = np.nan

df_nowe.isnull().sum().sum()/19250

0.0505974025974026

In [213]:
#KNN na wszystkich danych
wyn_knn_all=list()
n_range=range(1,30)
for i in n_range:
  kmni_all = KNNImputer(n_neighbors=i)
  kmni=kmni_all.fit_transform(df_nowe[cols])
  wyn_knn_all.append(np.sqrt(mean_squared_error(df_bezna[cols], kmni)))

wyn_knn_all_df=pd.DataFrame(np.column_stack([n_range, wyn_knn_all]), columns=["k","rmse"])

fig = px.line(wyn_knn_all_df, x="k", y="rmse", markers=True, line_shape='linear')

# Dodajemy etykiety i tytuł
fig.update_layout(
    xaxis_title='Liczba sąsiadów (k)',
    yaxis_title='Miara RMSE'
)

# Wyświetlamy wykres
fig.show()
fig.write_image("knn_all.png", width=1000, height=600, scale=4, format="png")

In [229]:
wyn_knn_all_df.iloc[wyn_knn_all_df["rmse"].idxmin()]

k       17.000000
rmse     0.138542
Name: 16, dtype: float64

In [211]:
# Inicjalizujemy słownik do przechowywania wyników
pacjenci_df_knn = {}
n_neighbors_range = range(1, 12)
wyn_knn_pac=list()

for n_neighbors in n_neighbors_range:
    knn_imp = KNNImputer(n_neighbors=n_neighbors)
    pacjenci_df_knn = np.concatenate([knn_imp.fit_transform(pacjent_df[cols]) for _, pacjent_df in df_nowe.groupby(by='PACJENT_NR')],axis=0)
    pacjenci_df_knn= pd.DataFrame(pacjenci_df_knn, columns=cols)
    wyn_knn_pac.append(np.sqrt(mean_squared_error(df_bezna[cols], pacjenci_df_knn)))

In [233]:
wyn_knn_pac_df=pd.DataFrame(np.column_stack([n_neighbors_range, wyn_knn_pac]), columns=["k","rmse"])

fig = px.line(wyn_knn_pac_df, x="k", y="rmse", markers=True, line_shape='linear')

# Dodajemy etykiety i tytuł
fig.update_layout(
    xaxis_title='Liczba sąsiadów (k)',
    yaxis_title='Miara RMSE'
)

# Wyświetlamy wykres
fig.show()
fig.write_image("knn_patient.png", width=1000, height=600, scale=4, format="png")


In [231]:

imputers = {
    "mean": Imputer(method="mean"),
    "median": Imputer(method="median"),
    "drift": Imputer(method="drift"),
    "backfill": Imputer(method="backfill"),
    "ffill": Imputer(method="pad"),
    "knn_p": KNNImputer(n_neighbors=2)
}

pacjenci_df = {}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for imp_name, imp in imputers.items():
        pacjenci_df[imp_name] = np.concatenate(
            [imp.fit_transform(pacjent_df[cols]) for _, pacjent_df in df_nowe.groupby(by='PACJENT_NR')],
            axis=0
        )
        pacjenci_df[imp_name] = pd.DataFrame(pacjenci_df[imp_name], columns=cols)

kmni_imputer = KNNImputer(n_neighbors=12)
pacjenci_df["knn_all"]=kmni_imputer.fit_transform(df_nowe[cols])


In [232]:
print("Podstawienie średniego wyniku pacjenta: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['mean']), 4)))
print("Podstawienie mediany wyników pacjenta: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['median']), 4)))
print("drift ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['drift']), 4)))
print("ffill ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['ffill']), 4)))
print("backfill ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['backfill']), 4)))
print("Metoda najbliższych sąsiadów k=2 na pacjentach: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df['knn_p']),4)))
print("Metoda najbliższych sąsiadów k=12 na wszystkich: ", str(np.round(mean_squared_error(df_bezna[cols], pacjenci_df["knn_all"]),4)))

Podstawienie średniego wyniku pacjenta:  0.066
Podstawienie mediany wyników pacjenta:  0.069
drift  0.0515
ffill  0.05
backfill  0.0577
Metoda najbliższych sąsiadów k=2 na pacjentach:  0.0244
Metoda najbliższych sąsiadów k=12 na wszystkich:  0.0197


### Uzupełnienie danych najlepszą metoda

In [245]:
#wypełnianie braków metodą K najbliższych sąsiadów
imputer = KNNImputer(n_neighbors=12)
df_pom=pd.DataFrame();

df_prepared = pd.DataFrame(imputer.fit_transform(df_pom), columns=df_pom.columns)
df_prepared


,PACJENT_NR,BADANIE_NR,ZGON,BETET,CO2TET,HCO3ACTE,HCO3STTE,O2SATTET,O2TET,PHTET,IONH
0,1.0,1.0,0.0,0.326087,2.21,0.850000,0.225,-2.48,-0.720,-0.810000,1.882698
1,1.0,2.0,0.0,0.086957,5.22,0.866667,0.200,0.86,4.880,-0.373333,1.425028
2,1.0,3.0,0.0,0.760870,3.10,1.250000,0.800,0.92,6.000,-1.060000,2.201643
3,1.0,4.0,0.0,0.869565,2.75,1.266667,0.675,0.62,1.440,-0.810000,1.882698
4,1.0,5.0,0.0,0.608696,2.61,1.083333,0.725,0.82,2.800,-0.880000,1.970160
...,...,...,...,...,...,...,...,...,...,...,...
2929,145.0,5.0,1.0,-0.717391,1.63,0.133333,-0.350,0.18,0.420,-1.130000,2.294287
2930,145.0,6.0,1.0,-0.891304,1.39,-0.016667,-0.450,0.76,1.640,-1.100000,2.254399
2931,145.0,7.0,1.0,-1.065217,1.71,-0.050000,-0.650,-0.06,0.316,-1.430000,2.708690
2932,145.0,8.0,1.0,-0.695652,1.15,0.066667,-0.275,0.62,1.600,-0.790000,1.857967


In [246]:
df_prepared.to_csv("gasometry_prepared_data.csv")